In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
import osmnx as ox

In [2]:
#north, south, east, west
G = ox.graph_from_place("Oradell,NJ,USA", network_type='walk')
#G=ox.graph_from_bbox(41.01132359438095, 40.90736914977194, -74.07244863958279, -73.97009723735901, network_type='walk')


In [3]:
nodes, edges = ox.utils_graph.graph_to_gdfs(G)
edges.head()

osmid  \
u         v          key                         
103093596 103243783  0                11588730   
          103293719  0                11588730   
          103285223  0    [11585865, 11562297]   
          6266877293 0                11562297   
103115864 103115866  0                11564531   

                                                                name  \
u         v          key                                               
103093596 103243783  0                                Delford Avenue   
          103293719  0                                Delford Avenue   
          103285223  0    [Glenside Court East, East Glenside Court]   
          6266877293 0                           Glenside Court East   
103115864 103115866  0                             Howard Court East   

                              highway  oneway   length  \
u         v          key                                 
103093596 103243783  0    residential   False   78.947   
          103293719  0    residential   False   78.947   
          103285223  0    residential   False  151.816   
          6266877293 0    residential   False   52.936   
103115864 103115866  0    residential   False   75.572   

                                                                   geometry  \
u         v          key                                                      
103093596 103243783  0    LINESTRING (-74.04112 40.94659, -74.04081 40.9...   
          103293719  0    LINESTRING (-74.04112 40.94659, -74.04143 40.9...   
          103285223  0    LINESTRING (-74.04112 40.94659, -74.04253 40.9...   
          6266877293 0    LINESTRING (-74.04112 40.94659, -74.04058 40.9...   
103115864 103115866  0    LINESTRING (-74.04174 40.94527, -74.04259 40.9...   

                         bridge  ref lanes service access  
u         v          key                                   
103093596 103243783  0      NaN  NaN   NaN     NaN    NaN  
          103293719  0      NaN  NaN   NaN     NaN    NaN  
          103285223  0      NaN  NaN   NaN     NaN    NaN  
          6266877293 0      NaN  NaN   NaN     NaN    NaN  
103115864 103115866  0      NaN  NaN   NaN     NaN    NaN

In [4]:
streets = edges.explode('highway')

In [5]:
streets['highway'].unique() 

array(['residential', 'tertiary', 'service', 'primary', 'secondary',
       'footway', 'secondary_link', 'path'], dtype=object)

In [6]:
# select only highway, osmid and geometry
streets = streets[['highway', 'osmid', 'geometry']]

In [7]:
# reset index and explode osmid
streets = streets.explode('osmid').reset_index(drop=True)

In [8]:
streets

,highway,osmid,geometry
0,residential,11588730,"LINESTRING (-74.04112 40.94659, -74.04081 40.9..."
1,residential,11588730,"LINESTRING (-74.04112 40.94659, -74.04143 40.9..."
2,residential,11585865,"LINESTRING (-74.04112 40.94659, -74.04253 40.9..."
3,residential,11562297,"LINESTRING (-74.04112 40.94659, -74.04253 40.9..."
4,residential,11562297,"LINESTRING (-74.04112 40.94659, -74.04058 40.9..."
...,...,...,...
1981,service,935428894,"LINESTRING (-74.03375 40.96103, -74.03370 40.9..."
1982,tertiary,1052968146,"LINESTRING (-74.03375 40.96103, -74.03140 40.9..."
1983,service,935428894,"LINESTRING (-74.03334 40.96163, -74.03343 40.9..."
1984,service,1111134819,"LINESTRING (-74.02994 40.95155, -74.02993 40.9..."


In [9]:
#save to shapefile
#streets.to_file('data/seattle-osm-roads/seattle-osm-roads.shp')

In [9]:
streets.explore(tiles='cartodbpositron', column='highway',cmap='Set2')

In [10]:
#read in geojson file for streets
ps_streets = gpd.read_file('data/source/streets.json')

In [11]:
ps_streets['osm_id'].nunique()

203

In [13]:

#drop avg_image_date and avg_label_date
ps_streets = ps_streets.drop(['avg_image_date', 'avg_label_date'], axis=1)

In [14]:
#select the ones in streets that are in ps_streets by osmid
streets = streets[streets['osmid'].isin(ps_streets['osm_id'])]

In [15]:
#grouby osmid and pick the first highway type
grouped = streets.groupby('osmid').first().reset_index()
#drop the geometry column
grouped = grouped.drop('geometry', axis=1)

In [16]:
grouped

,osmid,highway
0,11562297,residential
1,11564127,residential
2,11564531,residential
3,11564801,residential
4,11564836,residential
...,...,...
198,918422008,tertiary
199,970939924,secondary
200,970939925,secondary
201,1014487602,primary


In [17]:
#merge the two dataframes
df= ps_streets.merge(grouped, left_on='osm_id', right_on='osmid')

In [18]:
df = df.drop('osmid', axis=1)

In [19]:
df.explore(tiles='cartodbpositron', column='highway', cmap='Set2')

In [20]:
!mkdir data/oradell-osm-roads
df.to_file('data/oradell-osm-roads/ordell-osm-roads.shp')